In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
import os
from scipy.signal import find_peaks

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Read videos from the folder
# Get the video file names

video_files = os.listdir('videos')
video_files = [file for file in video_files if file.endswith('.mp4')]
print(video_files)

caps = []
for file in video_files:
    cap = cv2.VideoCapture('videos/' + file)
    if not cap.isOpened():
        print("Error opening video stream or file" + file)
    else:
        caps.append(cap)

    

['1.mp4', '2.mp4']


In [2]:
lowest_y = []

def _calculate_angle(p1, p2, p3):
    '''
    Calculate the angle between three points. As defined in report.
    '''
    a = np.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)
    b = np.sqrt((p2[0] - p3[0])**2 + (p2[1] - p3[1])**2)
    c = np.sqrt((p3[0] - p1[0])**2 + (p3[1] - p1[1])**2)
    angle = np.acos((b**2 + c**2 - a**2)/(2*b*c))
    return np.degrees(angle)

In [6]:
# Calculate lowest y value for each video by looking at local minima of ankle facing the camera
# Filter out innaccuracies by adding condition that peaks should be separated by at least 10 frames
# get the frame number of the lowest y value for each video

video_frames = []

for cap in caps:

    frames = []

    with mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:

        while cap.isOpened():
            success, image = cap.read()
            if not success:
                break

            # Flip the image horizontally for a later selfie-view display, and convert
            # the BGR image to RGB.
            image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            results = pose.process(image)

            # Draw the pose annotation on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            

            try:
                pose_landmarks = results.pose_landmarks.landmark

                # Get the coordinates
                left_ankle = [pose_landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, pose_landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_ankle = [pose_landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, pose_landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

                if left_ankle[0] < right_ankle[0]:
                    ankle = left_ankle

                else:
                    ankle = right_ankle

                frames.append(ankle[1])

            except:
                pass

    # Check for local minima
    # Add condition that peaks should be separated by at least 10 frames
    # Get the frame number of the lowest y value

    local_minima, _ = find_peaks(frames, distance=10)
    
    # Add contents of local_minima to video_frames
    video_frames.append(local_minima)

print(video_frames)




[array([], dtype=int64), array([], dtype=int64)]


In [4]:
# Use the frame number to get the lowest y value for each video



SyntaxError: incomplete input (190108765.py, line 57)